# **Mountain Car (Discrete case)**

References :

SARSA:  https://github.com/nicklasbekkevold/SARSA-mountaincar

SARSA (lambda) : https://github.com/dariopavllo/mountaincar-sarsa-lambda

Semi-Gradient SARSA : https://github.com/MikeS96/rl_openai/blob/master/Mountain%20Car.ipynb
http://www.incompleteideas.net/book/RLbook2018.pdf#page=267


PPO:    https://github.com/rossettisimone/PPO_MOUNTAINCAR_DISCRETE.

Reddit: www.reddit.com/r/reinforcementlearning/comments/axp63j/d_state_of_the_art_deeprl_still_struggles_to/.  
https://arxiv.org/pdf/1802.05054


Documentation pour la librairie tiles3 : http://incompleteideas.net/tiles/tiles3.html


Q-learning : https://www.kaggle.com/code/rezafazel63/mountain-car-is-a-classic-reinforcement-learning


  

# Google Colab configs

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

In [1]:
!pip install gym
!pip install gym[classic_control]
!pip install pyvirtualdisplay

     ---------------------------------------- 0.0/721.7 kB ? eta -:--:--
     - ----------------------------------- 20.5/721.7 kB 217.9 kB/s eta 0:00:04
     -- ---------------------------------- 41.0/721.7 kB 393.8 kB/s eta 0:00:02
     ---------------- --------------------- 317.4/721.7 kB 2.2 MB/s eta 0:00:01
     -------------------------------- ----- 614.4/721.7 kB 3.2 MB/s eta 0:00:01
     -------------------------------------- 721.7/721.7 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numpy-2.2.0-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached cloudpickle-3.1.0-py3-none-any.whl.metadata (7.0 kB)
Using cached cloudpickle-3.1.0-py3-none-any.whl (22 kB)
Usi


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/5.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.8 MB ? eta -:--:--
      --------------------------------------- 0.1/5.8 MB 573.4 kB/s eta 0:00:10
     -- ------------------------------------- 0.4/5.8 MB 2.4 MB/s eta 0:00:03
     -------- ------------------------------- 1.2/5.8 MB 5.4 MB/s eta 0:00:01
     ---------------- ----------------------- 2.4/5.8 MB 8.8 MB/s eta 0:00:01
     -------------------------- ------------- 3.9/5.8 MB 12.3 MB/s eta 0:00:01
     ---------------------------------- ----- 5.0/5.8 MB 14.0 MB/s eta 0:00:01
     ---------------------------------------  5.8/5.8 MB 16.1 MB/s eta 0:00:01
     ---------------------------------------- 5.8/5.8 MB 14.2 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Gettin

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [95 lines of output]
      
      
      WARNING, No "Setup" File Exists, Running "buildconfig/config.py"
      Using WINDOWS configuration...
      
      Making dir :prebuilt_downloads:
      Downloading... https://www.libsdl.org/release/SDL2-devel-2.0.16-VC.zip 13d952c333f3c2ebe9b7bc0075b4ad2f784e7584
      Unzipping :prebuilt_downloads\SDL2-devel-2.0.16-VC.zip:
      Downloading... https://www.libsdl.org/projects/SDL_image/release/SDL2_image-devel-2.0.5-VC.zip 137f86474691f4e12e76e07d58d5920c8d844d5b
      Unzipping :prebuilt_downloads\SDL2_image-devel-2.0.5-VC.zip:
      Downloading... https://www.libsdl.org/projects/SDL_ttf/release/SDL2_ttf-devel-2.0.15-VC.zip 1436df41ebc47ac36e02ec9bda5699e80ff9bd27
      Unzipping :prebuilt_downloads\SDL2_ttf-devel-2.0.15-VC.zip:
      Downloading... https://www.libsdl.org/projects/SDL_mixer/release/SDL2_mixer-devel-


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# !pip install gym pyvirtualdisplay > /dev/null 2>&1
# !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
# !apt-get update > /dev/null 2>&1
# !apt-get install cmake > /dev/null 2>&1
# !pip install --upgrade setuptools 2>&1
# !pip install ez_setup > /dev/null 2>&1
# !apt-get install -y xvfb


def wrap_env(env):
    env = RecordVideo(env, "./video")
    return env


def show_video():
    mp4list = glob.glob("video/*.mp4")
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, "r+b").read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(
            HTML(
                data="""<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>""".format(
                    encoded.decode("ascii")
                )
            )
        )
    else:
        print("Could not find video")


# A executer avant l'initialisation de env
# display = Display(visible=0, size=(1400, 900))
# display.start()
# env = wrap_env(gym.make('MountainCar-v0'))

# Libraries

In [4]:
import numpy as np
import gymnasium as gym

# from gym.wrappers.record_video import RecordVideo
# import glob
# import io
# import base64
# from IPython.display import HTML
# from pyvirtualdisplay import Display
# from IPython import display as ipythondisplay

# 1. Functions & Algorithms

## 1.1. Utils

In [5]:
import math
import random


class IHT:
    def __init__(self, sizeval):
        self.size = sizeval
        self.overfullCount = 0
        self.dictionary = {}

    def __str__(self):
        return (
            "Collision table: size: "
            + str(self.size)
            + " overfullCount: "
            + str(self.overfullCount)
            + " dictionary: "
            + str(len(self.dictionary))
        )

    def count(self):
        return len(self.dictionary)

    def fullp(self):
        return len(self.dictionary) >= self.size

    def getindex(self, obj, readonly=False):
        if obj in self.dictionary:
            return self.dictionary[obj]
        elif readonly:
            return None
        size = self.size
        if len(self.dictionary) >= size:
            if self.overfullCount == 0:
                print("IHT full, starting to allow collisions")
            self.overfullCount += 1
            return hash(obj) % self.size
        else:
            self.dictionary[obj] = len(self.dictionary)
            return self.dictionary[obj]


def hashcoords(coordinates, m, readonly=False):
    if isinstance(m, IHT):
        return m.getindex(tuple(coordinates), readonly)
    if isinstance(m, int):
        return hash(tuple(coordinates)) % m
    if m is None:
        return coordinates


def get_tiles(ihtORsize, numtilings, floats, ints=[], readonly=False):
    qfloats = [int(math.floor(f * numtilings)) for f in floats]
    tiles = []
    for tiling in range(numtilings):
        tilingX2 = tiling * 2
        coords = [tiling]
        b = tiling
        for q in qfloats:
            coords.append((q + b) // numtilings)
            b += tilingX2
        coords.extend(ints)
        tiles.append(hashcoords(coords, ihtORsize, readonly))
    return tiles

In [6]:
POSITION_MIN = -1.2
POSITION_MAX = 0.5
VELOCITY_MIN = -0.07
VELOCITY_MAX = 0.07

iht_size = 2024
num_tilings = 8
num_tiles = 8

iht = IHT(2024)


def mc_tile_encoding(state):
    """
    Tile encoding function for states in mountain car gymnasium problem

    Args:
        - state (tuple): (position, velocity)

    Returns:
        - The list of the tiles corresponding to the given state
    """
    # Extract the position and velocity from the gymnasium state
    position, velocity = state

    # Scale position and velocity by multiplying the inputs of each by their scale
    position_scale = num_tiles / (POSITION_MAX - POSITION_MIN)
    velocity_scale = num_tiles / (VELOCITY_MAX - VELOCITY_MIN)

    # Obtain active tiles for current position and velocity
    tiles = get_tiles(
        iht, num_tilings, [position * position_scale, velocity * velocity_scale]
    )

    return np.array(tiles)


print("Tiles activées :", mc_tile_encoding((-1.0, 0.01)))

Tiles activées : [0 1 2 3 4 5 6 7]


## 1.2. Agents

### 1.2.1. SARSA Agent

In [38]:
# SARSA : Apprentissage on-policy avec mise à jour incrémentale des Q-valeurs.

# alpha : Taux d’apprentissage.
# gamma : Facteur de discount.
# epsilon : Paramètre d’exploration.
env = gym.make("MountainCar-v0")
num_actions = env.action_space.n


def sarsa_mountain_car(episodes=500, alpha=0.1, gamma=0.99, epsilon=0.1):

    def get_Q_value(Q, state_tiles, action):
        """
        For one state, summing on every values of corresponding tiles, with default = 0 if no value.
        """
        return sum(Q.get((tile, action), 0) for tile in state_tiles)

    def choose_action(Q, state_tiles, epsilon):
        if np.random.rand() < epsilon:
            return np.random.choice(num_actions)  # Exploration
        else:
            # Exploitation: Choose action with maximum Q-value
            q_values = [get_Q_value(Q, state_tiles, a) for a in range(num_actions)]
            return np.argmax(q_values)

    # Initialize Q-values (dictionary with default value 0)
    Q = {}

    for episode in range(episodes):
        state, _ = env.reset()
        state_tiles = mc_tile_encoding(state)
        action = choose_action(Q, state_tiles, epsilon)

        done, time_over = False, False
        total_reward = 0

        while not (done or time_over):
            next_state, reward, done, time_over, _ = env.step(action)
            next_state_tiles = mc_tile_encoding(next_state)
            next_action = choose_action(Q, next_state_tiles, epsilon)

            # Compute the SARSA update
            q_current = get_Q_value(Q, state_tiles, action)
            q_next = get_Q_value(Q, next_state_tiles, next_action) if not done else 0
            target = reward + gamma * q_next
            error = target - q_current

            # Update Q-values for all tiles
            for tile in state_tiles:
                Q[(tile, action)] = Q.get((tile, action), 0) + alpha * error

            # Move to the next state and action
            state_tiles = next_state_tiles
            action = next_action
            total_reward += reward

        print(f"Episode {episode + 1}: Total Reward = {total_reward}")

    return Q

In [21]:
class SarsaAgent:

    def __init__(
        self,
        env: gym.Env,
        encode_fct,
        nb_actions=None,
    ):
        """
        - env (gymnasium.Env): The environment to train on.
        - nb_actions (int): Number of possible actions.
        - encode_fct (callable): Function to encode the state into features (e.g., tile coding).
        """
        self.env = env
        self.nb_actions = env.action_space.n if nb_actions == None else nb_actions
        self.encode_fct = encode_fct
        self.q = {}

    def init(self):
        """Initialize q values"""
        self.q = {}

    def get_q_value(self, q, state, action):
        """Compute Q-value for a state-action pair by summing over the tiles."""
        return sum(q.get((tile, action), 0) for tile in self.encode_fct(state))

    def choose_action(self, q, state, epsilon):
        """Choose an action using an epsilon-greedy policy."""
        if np.random.rand() < epsilon:
            return np.random.choice(self.nb_actions)  # Exploration
        else:
            # Exploitation: Select the action with the highest Q-value
            q_values = [self.get_q_value(q, state, a) for a in range(self.nb_actions)]
            return np.argmax(q_values)

    def train(
        self,
        nb_episodes,
        alpha=0.1,
        gamma=0.99,
        epsilon=0.1,
        use_glei=False,
    ):
        """
        SARSA algorithm for on-policy reinforcement learning.

        Args:
            - nb_episodes (int): Number of episodes to train for.
            - alpha (float): Learning rate for updating Q-values.
            - gamma (float): Discount factor for future rewards.
            - epsilon (float): Initial exploration rate for epsilon-greedy policy.
            - use_glei (bool): Whether to use a decaying epsilon (GLEI policy).
            - decay_rate (float): Decay rate for epsilon in GLEI policy.
            - min_epsilon (float): Minimum epsilon value in GLEI policy.

        Returns:
            - rewards_historic (list): History of rewards across episodes.
        """
        rewads_historic = []
        self.init()

        for episode in range(nb_episodes):
            state, _ = self.env.reset()
            action = self.choose_action(self.q, state, epsilon)

            task_completed, episode_over = False, False
            total_reward = 0

            while not (task_completed or episode_over):
                next_state, reward, task_completed, episode_over, _ = self.env.step(
                    action
                )
                next_action = self.choose_action(self.q, next_state, epsilon)

                # Compute the SARSA update
                q_current = self.get_q_value(self.q, state, action)
                q_next = (
                    self.get_q_value(self.q, next_state, next_action)
                    if not (task_completed or episode_over)
                    else 0
                )
                target = reward + gamma * q_next
                error = target - q_current

                # Update Q-values for all tiles
                for tile in self.encode_fct(state):
                    self.q[(tile, action)] = (
                        self.q.get((tile, action), 0) + alpha * error
                    )

                # Move to the next state and action
                state = next_state
                action = next_action
                total_reward += reward

            rewads_historic.append(total_reward)
            print(f"Episode {episode + 1}: Total Reward = {total_reward}")

        return rewads_historic

    def evaluate_agent(self, nb_trainings, nb_episodes):
        """
        Train the agent multiple times in order to evaluate its training parameters with mean and standard deviation over episodes
        """

In [9]:
def sarsa(
    env,
    nb_actions,
    encode_fct,
    nb_episodes=500,
    alpha=0.1,
    gamma=0.99,
    epsilon=0.1,
    use_glei=False,
    decay_rate=0.2,
    min_epsilon=0.05,
):
    """
    SARSA algorithm for on-policy reinforcement learning.

    Args:


    Returns:
        - q (dict): The Q-values represented as a dictionary.
        - rewards_historic (list): History of rewards across episodes.
    """
    # Initialize Q-values as a dictionary with default value 0
    q = {}
    rewards_historic = []
    count_decay = 1

    for episode in range(nb_episodes):
        # Initialize environment and encode initial state
        state = env.reset()[0]
        state_tiles = encode_fct(state)
        action = choose_action_sarsa(q, state_tiles, nb_actions, epsilon)

        total_reward = 0
        terminated = False

        while not terminated:
            # Take a step in the environment
            next_state, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward

            # Encode the next state and choose the next action
            next_state_tiles = encode_fct(*next_state)
            next_action = choose_action_sarsa(q, next_state_tiles, nb_actions, epsilon)

            # SARSA Q-value update
            q_current = get_q_value_sarsa(q, state_tiles, action)
            q_next = (
                get_q_value_sarsa(q, next_state_tiles, next_action)
                if not terminated
                else 0
            )
            q_target = reward + gamma * q_next
            error = q_target - q_current

            # Update Q-values for all tiles in the state representation
            for tile in state_tiles:
                q[(tile, action)] = q.get((tile, action), 0) + alpha * error

            # Move to the next state and action
            state_tiles = next_state_tiles
            action = next_action

        # Track total reward for the episode
        rewards_historic.append(total_reward)

        # Decay epsilon if GLEI policy is enabled
        if use_glei and episode > decay_rate * count_decay * nb_episodes:
            epsilon = max(epsilon * (1 - decay_rate * count_decay), min_epsilon)
            count_decay += 1

        print(f"Episode {episode + 1}: Total Reward = {total_reward}")

    return q, rewards_historic

In [39]:
# Run SARSA
env = gym.make("MountainCar-v0")
num_actions = env.action_space.n
Q = sarsa_mountain_car(episodes=1000, alpha=0.1, gamma=0.99, epsilon=0.01)

Episode 1: Total Reward = -200.0
Episode 2: Total Reward = -200.0
Episode 3: Total Reward = -200.0
Episode 4: Total Reward = -200.0
Episode 5: Total Reward = -200.0
Episode 6: Total Reward = -200.0
Episode 7: Total Reward = -200.0
Episode 8: Total Reward = -200.0
Episode 9: Total Reward = -200.0
Episode 10: Total Reward = -200.0
Episode 11: Total Reward = -200.0
Episode 12: Total Reward = -200.0
Episode 13: Total Reward = -200.0
Episode 14: Total Reward = -200.0
Episode 15: Total Reward = -200.0
Episode 16: Total Reward = -199.0
Episode 17: Total Reward = -200.0
Episode 18: Total Reward = -200.0
Episode 19: Total Reward = -200.0
Episode 20: Total Reward = -200.0
Episode 21: Total Reward = -172.0
Episode 22: Total Reward = -200.0
Episode 23: Total Reward = -200.0
Episode 24: Total Reward = -188.0
Episode 25: Total Reward = -154.0
Episode 26: Total Reward = -180.0
Episode 27: Total Reward = -152.0
Episode 28: Total Reward = -200.0
Episode 29: Total Reward = -200.0
Episode 30: Total Rewar

In [25]:
v0_agent = SarsaAgent(gym.make("MountainCar-v0"), mc_tile_encoding)

rewards_historic = v0_agent.train(
    nb_episodes=1000, alpha=0.1, gamma=0.99, epsilon=0.001
)

Episode 1: Total Reward = -200.0
Episode 2: Total Reward = -200.0
Episode 3: Total Reward = -200.0
Episode 4: Total Reward = -200.0
Episode 5: Total Reward = -200.0
Episode 6: Total Reward = -200.0
Episode 7: Total Reward = -200.0
Episode 8: Total Reward = -200.0
Episode 9: Total Reward = -200.0
Episode 10: Total Reward = -200.0
Episode 11: Total Reward = -200.0
Episode 12: Total Reward = -200.0
Episode 13: Total Reward = -200.0
Episode 14: Total Reward = -200.0
Episode 15: Total Reward = -200.0
Episode 16: Total Reward = -200.0
Episode 17: Total Reward = -200.0
Episode 18: Total Reward = -200.0
Episode 19: Total Reward = -200.0
Episode 20: Total Reward = -200.0
Episode 21: Total Reward = -200.0
Episode 22: Total Reward = -200.0
Episode 23: Total Reward = -200.0
Episode 24: Total Reward = -200.0
Episode 25: Total Reward = -200.0
Episode 26: Total Reward = -200.0
Episode 27: Total Reward = -200.0
Episode 28: Total Reward = -200.0
Episode 29: Total Reward = -159.0
Episode 30: Total Rewar

In [43]:
def test_policy(Q, num_episodes=10, render=False):
    render = "humane" if render else None
    env = gym.make("MountainCar-v0", render_mode=None)
    total_rewards = []

    for episode in range(num_episodes):
        state = env.reset()[0]
        time_over = False
        done = False
        total_reward = 0

        while not (done or time_over):

            # Obtenir les tiles pour l'état actuel
            state_tiles = mc_tile_encoding(state)

            # Choisir l'action qui maximise Q(s, a)
            q_values = [
                sum(Q.get((tile, action), 0) for tile in state_tiles)
                for action in range(env.action_space.n)
            ]
            action = np.argmax(q_values)

            # Exécuter l'action choisie
            state, reward, done, time_over, _ = env.step(action)
            # position, velocity = next_state

            total_reward += reward

        total_rewards.append(total_reward)
        print(f"Episode {episode + 1}: Total Reward = {total_reward}")

    env.close()
    print(
        f"Average Total Reward over {num_episodes} episodes: {np.mean(total_rewards)}"
    )

In [121]:
env.reset()

(array([-0.54168844,  0.        ], dtype=float32), {})

In [28]:
v0_agent.q

{(np.int64(8), np.int64(0)): -8.22598552316601,
 (np.int64(9), np.int64(0)): -8.288832176430622,
 (np.int64(10), np.int64(0)): -8.075001368221631,
 (np.int64(11), np.int64(0)): -8.572339013826811,
 (np.int64(17), np.int64(0)): -8.975882686376794,
 (np.int64(30), np.int64(0)): -8.52300255648422,
 (np.int64(39), np.int64(0)): -9.011724711784177,
 (np.int64(15), np.int64(0)): -8.230545938347266,
 (np.int64(16), np.int64(0)): -7.585770497722181,
 (np.int64(16), np.int64(1)): -9.130089479254526,
 (np.int64(9), np.int64(1)): -8.970096801681484,
 (np.int64(10), np.int64(1)): -9.01209629533347,
 (np.int64(11), np.int64(1)): -8.616638257382926,
 (np.int64(17), np.int64(1)): -8.376646200198138,
 (np.int64(30), np.int64(1)): -8.697831852756996,
 (np.int64(39), np.int64(1)): -8.683530244209994,
 (np.int64(15), np.int64(1)): -9.895295480851699,
 (np.int64(16), np.int64(2)): -8.925532828349576,
 (np.int64(9), np.int64(2)): -8.716576897973766,
 (np.int64(10), np.int64(2)): -8.995065427238766,
 (np.in

In [52]:
# Tester la politique optimale
test_policy(Q, num_episodes=10, render=False)

Episode 1: Total Reward = -87.0
Episode 2: Total Reward = -146.0
Episode 3: Total Reward = -154.0
Episode 4: Total Reward = -149.0
Episode 5: Total Reward = -146.0
Episode 6: Total Reward = -145.0
Episode 7: Total Reward = -149.0
Episode 8: Total Reward = -83.0
Episode 9: Total Reward = -90.0
Episode 10: Total Reward = -148.0
Average Total Reward over 10 episodes: -129.7


In [50]:
# Tester la politique optimale
test_policy(v0_agent.q, num_episodes=10, render=False)

Episode 1: Total Reward = -142.0
Episode 2: Total Reward = -136.0
Episode 3: Total Reward = -175.0
Episode 4: Total Reward = -131.0
Episode 5: Total Reward = -173.0
Episode 6: Total Reward = -133.0
Episode 7: Total Reward = -145.0
Episode 8: Total Reward = -99.0
Episode 9: Total Reward = -140.0
Episode 10: Total Reward = -88.0
Average Total Reward over 10 episodes: -136.2


# 2. Experimentations

In [31]:
env = gym.make(
    "MountainCar-v0",
    render_mode="human",  # Cas discret
)

env.reset()
print(f"State space: {env.observation_space}")
print(f"Action space: {env.action_space}")

State space: Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
Action space: Discrete(3)


In [520]:
sum = 0
for i in range(201):
    print(i)
    a = env.step(2)
    sum += a[1]
    print(a)
    print(sum)

0
(array([-0.5598817 ,  0.00128113], dtype=float32), -1.0, False, False, {})
-1.0
1
(array([-0.55732894,  0.00255272], dtype=float32), -1.0, False, False, {})
-2.0
2
(array([-0.55352366,  0.00380526], dtype=float32), -1.0, False, False, {})
-3.0
3
(array([-0.5484943,  0.0050294], dtype=float32), -1.0, False, False, {})
-4.0
4
(array([-0.54227835,  0.00621594], dtype=float32), -1.0, False, False, {})
-5.0
5
(array([-0.53492236,  0.00735597], dtype=float32), -1.0, False, False, {})
-6.0
6
(array([-0.5264815 ,  0.00844088], dtype=float32), -1.0, False, False, {})
-7.0
7
(array([-0.51701903,  0.0094625 ], dtype=float32), -1.0, False, False, {})
-8.0
8
(array([-0.50660586,  0.01041315], dtype=float32), -1.0, False, False, {})
-9.0
9
(array([-0.4953201 ,  0.01128576], dtype=float32), -1.0, False, False, {})
-10.0
10
(array([-0.48324618,  0.01207392], dtype=float32), -1.0, False, False, {})
-11.0
11
(array([-0.47047415,  0.01277202], dtype=float32), -1.0, False, False, {})
-12.0
12
(array([-0

In [514]:
sum

-115.0

### Code Pierre

In [ ]:
def on_policy_monte_carlo(
    env,
    nb_states,
    nb_actions,
    encode_fct,
    nb_episodes=20000,
    gamma=1,
    epsilon=0.1,
    use_glei=False,
    decay_rate=0.2,
    min_epsilon=0.05,
):
    """
    This algorithm goal is to find the optimal policy in a episodic environement with reasonable number of states and actions
    Convergence guaranteed in this paper: https://sites.ualberta.ca/~szepesva/papers/sarsa98.ps.pdf

    Repeat for the given number of episodes:
        Simulate an episode with the given policy
        Compute the accumulated rewards with discount factor (gamma): G
        For every visited q(s, a) in the current episode:
            If first visit of (s, a) is at iteration t:
                acc(s, a) += Gt
                count(s, a) += 1
                q(s, a) = acc(s, a) / count(s, a)
            For every state in the current episode:
                update policy(s, a) with an epsilon greedy strategy or GLEI strategy

    Args:
        - env (gymnasium.Env): a gymnasium environment
        - nb_states (int): the number of states for the given env
        - nb_actions (int): the number of actions for the given env
        - encode_fct (callable): the function to encode our state.
        - nb_episodes (int): number of episodes simulated for learning
        - gamma (float): discount factor for updating states values
        - epsilon (float): initial exploration factor, in [0, 1], if use_gley = True, represent the starting epsilon
        - use_glei (bool): if True, use GLEI policy (decaying epsilon across episodes); if False, use simple epsilon-greedy
        - decay_rate (float): Every time num_episode > nb_episode * (decay_rate *i) then epsilon = epsilon * (1 - decay_rate) * i. Where i is the number of time we decayed epsilon
        - min_epsilon (float): the minimum decayed epsilon when using a GLEI policy

    Returns:
        - policy (numpy.array): the learnt policy
        - q (numpy.array): the Q-values
        - rewards_historic (list): the historic of rewards during training
    """
    # Initialize algorithm variables
    q = np.zeros(
        shape=(nb_states, nb_actions)
    )  # (state, action) values through every episodes generated
    count = np.zeros(
        shape=(nb_states, nb_actions)
    )  # Counting each (s, a) events through every episodes generated
    acc = np.zeros(
        shape=(nb_states, nb_actions)
    )  # (state, action) cumulated values through every episodes generated
    policy = np.full(
        shape=(nb_states, nb_actions), fill_value=1 / nb_actions
    )  # Actions probabilities for each state. Initialized as a uniform policy
    rewards_historic = []
    count_decay = 1  # Decay rate factor

    for num_episode in range(nb_episodes):
        # print(f"Computing ep {num_episode}")

        # Initialize environment
        game_state = env.reset()[
            0
        ]  # A tuple (player score, opponent score, ace (1) or not (0))
        terminated = False
        G = np.empty((0, 1))  # Cumulated rewards of the current episode
        moves_historic = np.empty(
            (0, 2), dtype=int
        )  # Historic of (state, action) of the current episode

        # Step 1: Simulate an episode
        while not terminated:
            # Draw an action according to the current policy
            state_index = encode_fct(game_state)
            action = draw(policy, state_index)

            # Update environment with the chosen action
            env_state = env.step(action)
            game_state, G, terminated = (
                env_state[0],
                np.append(G, env_state[1]),
                env_state[2],
            )

            # Update current historic if current (state, action) was not visited yet
            moves_historic = np.append(moves_historic, [[state_index, action]], axis=0)

        # Update rewards_historic with the total reward of the episode
        rewards_historic.append(np.sum(G))

        # Step 2: Compute G, the cumulative rewards for each state
        gamma_powers = gamma ** np.arange(len(G))
        for i in range(len(G) - 1):
            G[i] = np.sum(G[i:] * gamma_powers[: len(G[i:])])

        # Step 3: For every (s, a) visited: Update q(s, a)
        # Extracting first visit for every (s, a)
        first_visit_states_action, first_visit_indices = np.unique(
            moves_historic, axis=0, return_index=True
        )
        first_visit_indices = np.sort(first_visit_indices)

        # Extract relevant states, actions, and rewards for first visits
        states = moves_historic[first_visit_indices, 0]
        actions = moves_historic[first_visit_indices, 1]
        rewards = G[first_visit_indices]

        # Update acc, count, and q arrays
        np.add.at(acc, (states, actions), rewards)  # Accumulate rewards
        np.add.at(count, (states, actions), 1)  # Increment count
        q = np.divide(acc, count, where=(count != 0), out=np.zeros_like(acc))

        # Step 4: Update policy with the current Q-values
        # Update epsilon if we use GLEI policy
        if (use_glei) & (num_episode > (decay_rate * count_decay) * nb_episodes):
            epsilon = max(epsilon * (1 - (decay_rate * count_decay)), min_epsilon)
            count_decay += 1
        policy = update_eps_greedy_policy(epsilon=epsilon, q=q)

    if use_glei:
        print(f"Final epsilon: {epsilon}")
    return policy, q, rewards_historic